In [2]:
import os
spark_version = 'spark-3.2.1'
os.environ['SPARK_VERSION']=spark_version

In [3]:
# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
#!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!wget -q https://downloads.apache.org/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Get:4 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [787 kB]
Get:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelea

In [4]:
# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

In [5]:
# Start a SparkSession
import findspark
findspark.init()

In [6]:
!wget https://jdbc.postgresql.org/download/postgresql-42.3.6.jar

--2022-05-26 23:21:57--  https://jdbc.postgresql.org/download/postgresql-42.3.6.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1041072 (1017K) [application/java-archive]
Saving to: ‘postgresql-42.3.6.jar’

postgresql-42.3.6.j 100%[===================>]   1017K  --.-KB/s    in 0.1s    

2022-05-26 23:21:58 (6.72 MB/s) - ‘postgresql-42.3.6.jar’ saved [1041072/1041072]



In [7]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.3.6.jar").getOrCreate()

In [8]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url="https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Software_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
software_data = spark.read.csv(SparkFiles.get("amazon_reviews_us_Software_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)

# Show DataFrame
software_data.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   42605767|R3EFW2STIYIY0I|B00MUTIDKI|     248732228|McAfee 2015 Inter...|        Software|          1|            2|          2|   N|                Y|I was very disapp...|I was very disapp...| 2015-08-31|
|         US|   51771800|R12NR0R5A9F7FT|B00EPACNUG|     531462352|Hallmark Card Stu...|        Software|          5|    

In [9]:
#Count the number of records (rows) in the dataset.
print(software_data.count())

341931


In [10]:
#**Transform** the dataset to fit the tables in the [schema file](../Resources/schema.sql). Be sure the DataFrames match in data type and in column name.
software_data.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



In [11]:
#create dataframe for CREATE TABLE review_id_table (
#  review_id TEXT PRIMARY KEY NOT NULL,
#  customer_id INTEGER,
#  product_id TEXT,
#  product_parent INTEGER,
#  review_date DATE -- this should be in the formate yyyy-mm-dd
#);

review_id_table_data = software_data.select(["review_id", "customer_id", "product_id", "product_parent", "review_date"])
review_id_table_data.show(5)

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R3EFW2STIYIY0I|   42605767|B00MUTIDKI|     248732228| 2015-08-31|
|R12NR0R5A9F7FT|   51771800|B00EPACNUG|     531462352| 2015-08-31|
|R1LSH74R9XAP59|   16053526|B00164AZA4|     473982505| 2015-08-31|
|R1QXUNTF76K7L6|   15319481|B00E6LIEFM|     189774198| 2015-08-31|
|R2F7DR75PS8NKT|    1441820|B00VWEBG06|     852470365| 2015-08-31|
+--------------+-----------+----------+--------------+-----------+
only showing top 5 rows



In [12]:
#-- This table will contain only unique values
#CREATE TABLE products (
#  product_id TEXT PRIMARY KEY NOT NULL UNIQUE,
#  product_title TEXT
#);
products_data = software_data.select(["product_id", "product_title"])
products_data.show(5)

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00MUTIDKI|McAfee 2015 Inter...|
|B00EPACNUG|Hallmark Card Stu...|
|B00164AZA4| Search and Rescue 4|
|B00E6LIEFM|      Quickbooks Pro|
|B00VWEBG06|Windows 7 Profess...|
+----------+--------------------+
only showing top 5 rows



In [13]:
#-- Customer table for first data set
#CREATE TABLE customers (
#  customer_id INT PRIMARY KEY NOT NULL UNIQUE,
#  customer_count INT
#);

customer_count = software_data.select(["customer_id"]).count()
customer_count


341931

In [14]:
customers_data = software_data.groupBy('customer_id').count()
customers_data_counts = customers_data.withColumnRenamed('count', 'customers_count')
customers_data_counts.show()

+-----------+---------------+
|customer_id|customers_count|
+-----------+---------------+
|   15634680|              1|
|   44696507|              1|
|   19901367|              1|
|   11337682|              1|
|   52021773|              1|
|   29916198|              1|
|   42030944|              1|
|   52765209|              1|
|   13519509|              2|
|   39320350|              5|
|   23249582|              1|
|   46351928|              3|
|   44179620|              1|
|     637252|              1|
|   34038793|              1|
|   51136490|              1|
|   49354815|              1|
|     149761|              2|
|   43333798|              1|
|    4657027|              1|
+-----------+---------------+
only showing top 20 rows



In [15]:
#-- vine table
#CREATE TABLE vine_table (
#  review_id TEXT PRIMARY KEY,
#  star_rating INTEGER,
#  helpful_votes INTEGER,
#  total_votes INTEGER,
#  vine TEXT
#);

vine_data = software_data.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine"])
vine_data.show(5)

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R3EFW2STIYIY0I|          1|            2|          2|   N|
|R12NR0R5A9F7FT|          5|            0|          0|   N|
|R1LSH74R9XAP59|          2|            0|          1|   N|
|R1QXUNTF76K7L6|          2|            0|          0|   N|
|R2F7DR75PS8NKT|          5|            0|          0|   N|
+--------------+-----------+-------------+-----------+----+
only showing top 5 rows



In [16]:
 #* **Load** the DataFrames that correspond to tables into an RDS instance. **Note:** This process can take up to 10 minutes for each. Be sure that everything is correct before uploading.
mode="append"
jdbc_url = "jdbc:postgresql://database-1.cloe6i70wwpd.us-east-1.rds.amazonaws.com:5432/PR1"
#jdbc_url = "jdbc:postgresql://192.168.86.77:5432/productreview"
config = {"user":"postgres",
          "password": "<password>",
          "driver":"org.postgresql.Driver"}

In [20]:
# Write review_id_table_df table RDS

review_id_table_data.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [21]:
# Write products_data  to RDS
products_data = products_data.dropDuplicates()
products_data.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [23]:
# Write DataFrame to customers table in RDS

customers_data_counts.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [24]:
# Write vine_data to customers table in RDS

vine_data.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)